For this homework, make sure that you format your notbook nicely and cite all sources in the appropriate sections. Programmatically generate or embed any figures or graphs that you need.

Names: __YOUR NAMES HERE__

Step 1: Train your own word embeddings
--------------------------------

(describe the provided dataset that you have chosen here)

Describe what data set you have chosen to compare and contrast with the your chosen provided dataset. Make sure to describe where it comes from and it's general properties.

(describe your dataset here)

In [23]:
# import your libraries here
import nltk
import numpy as np
import random
from collections import Counter
from nltk.tokenize import RegexpTokenizer
# libs used for preprocessing
from gensim.parsing.preprocessing import stem_text, remove_stopwords, strip_punctuation
from nltk import ngrams
# libs used for file reading and parsing
from csv import reader
from csv import writer

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zhanghaotian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### a) Train embeddings on GIVEN dataset

In [4]:
def parse_csv(training_file_path: str, select_column:int, percentage: float) -> list[str]:
  """This function is used to parse input lines
  and returns a the provided percent of data.

  Args:
      lines (List[str]): list of lines
      percentage (int): percent of the dataset needed
      select_column (int): column to be selected from the dataset
  Returns:
      List[str]: lines (percentage of dataset)
  """
  sentences = []
  with open(training_file_path, "r", encoding="utf8", errors="ignore") as csvfile:
    csv_reader = reader(csvfile)
    #skipping header
    header = next(csv_reader)

    # line_length = len(list(csv_reader_copy))
   
    if header != None:
      for row in csv_reader:
        sentences.append(row[select_column])

  random.shuffle(sentences)
  to_return = sentences[:int(len(sentences)*percentage)]

  return to_return

In [16]:
def preprocessing(running_lines: list[str], ngrams: int) -> list[list[str]]:
  """This function takes in the running test and return back the
  preprocessed text. Four tasks are done as part of this:
    1. lower word case
    2. remove stopwords
    3. remove punctuation
    4. Add - <s> and </s> for every sentence

  Args:
      running_lines (List[str]): list of lines

  Returns:
      List[List[str]]: list of sentences where each sentence is broken
                        into list of words.
  """
  preprocessed_lines = []
  tokenizer = RegexpTokenizer(r'\w+')
  for line in running_lines:
    lower_case_data = line.lower()
    data_without_stop_word = remove_stopwords(lower_case_data)
    data_without_punct = strip_punctuation(data_without_stop_word)
    processed_data = tokenizer.tokenize(data_without_punct)
    for i in range(1, ngrams):
      processed_data.insert(0,"<s>")
    processed_data.append("</s>")
    preprocessed_lines.append(processed_data)
  return preprocessed_lines

In [18]:
halfspooky = parse_csv("train.csv", 1, 0.5)
halfspooky = preprocessing(halfspooky, 3)
random.shuffle(halfspooky)

In [28]:
financial_set = parse_csv("financial_data.csv", 0, 1)
financial_set = preprocessing(financial_set, 3)
random.shuffle(financial_set)
list_to_write = []
for i in range(len(financial_set)):
  list_to_write.append([financial_set[i]])

In [33]:
testset = parse_csv("financial_data_training_set.csv", 0, 1)

In [42]:
def write_into_csv(data: list[list[str]], file_name: str):
  """This function takes in the data and writes it into a csv file.

  Args:
      data (List[List[str]]): list of sentences where each sentence is broken
                        into list of words.
      file_name (str): name of the file to be written
  """
  with open(file_name, 'w') as f:
    csv_writer = writer(f)
    csv_writer.writerows(data)
  f.close()

def read_from_csv(file_name: str):
  data = []
  with open(file_name, 'r') as f:
    csv_reader = reader(f)
    for row in csv_reader:
      data.append(row)
  f.close()
  return data

In [44]:
write_into_csv(financial_set, "financial_data_training_set.csv", )
write_into_csv(halfspooky, "half_spooky_training_set.csv")

In [45]:
from gensim.models import Word2Vec

# The dimension of word embedding. 
# This variable will be used throughout the program
# you may vary this as you desire
EMBEDDINGS_SIZE = 200

# Train the Word2Vec model from Gensim. 
# Below are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
# sg = 1
# window = 5
# vector_size = EMBEDDINGS_SIZE
# min_count = 1

w2v_model_half_spooky = Word2Vec(halfspooky, vector_size=EMBEDDINGS_SIZE, window=5, min_count=1, sg=1, sorted_vocab=1, workers=4)
w2v_model_financial = Word2Vec(financial_set, vector_size=EMBEDDINGS_SIZE, window=5, min_count=1, sg=1, sorted_vocab=1, workers=4)


In [46]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
print('Vocab size {}'.format(len(w2v_model_half_spooky.wv.key_to_index)))
print('Vocab size {}'.format(len(w2v_model_financial.wv.key_to_index)))

Vocab size 19105
Vocab size 11266


In [47]:
# You can save file in txt format, then load later if you wish.
w2v_model_half_spooky.wv.save_word2vec_format('half_spooky_embeddings.txt', binary=False)
w2v_model_financial.wv.save_word2vec_format('financial_embeddings.txt', binary=False)

### b) Train embedding on YOUR dataset

What text-normalization and pre-processing did you do and why? __YOUR ANSWER HERE__

Step 2: Evaluate the differences between the word embeddings
----------------------------

(make sure to include graphs, figures, and paragraphs with full sentences)

## Write down your analysis:

Cite your sources:
-------------

Step 3: Feedforward Neural Language Model
--------------------------

### a) First, encode  your text into integers

In [9]:
# Importing utility functions from Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

# The size of the ngram language model you want to train
# change as needed for your experiments
NGRAM = 3

# Initializing a Tokenizer
# It is used to vectorize a text corpus. Here, it just creates a mapping from 
# word to a unique index. (Note: Indexing starts from 0)
# Example:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(data)
# encoded = tokenizer.texts_to_sequences(data)
def read_lines_from_generated_data(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            data.append(line.split())
    return data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(halfspooky)
encoded = tokenizer.texts_to_sequences(halfspooky)

2023-03-21 19:30:33.339994: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### b) Next, prepare your sequences from text

#### Fixed ngram based sequences 

The training samples will be structured in the following format. 
Depending on which ngram model we choose, there will be (n-1) tokens 
in the input sequence (X) and we will need to predict the nth token (Y)

            X,						  y
    this,    process               however
    process, however               afforded
    however, afforded	           me

In [10]:
def generate_ngram_training_samples(encoded: list, ngram:int) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    training_samples = []
    for line in encoded:
        for i in range(len(line) - ngram + 1):
            training_samples.append(line[i:i+ngram])
    return training_samples


In [11]:
generated_ngram_samples = generate_ngram_training_samples(encoded, NGRAM)
generated_ngram_samples[:-10]

[[1, 6, 2010],
 [6, 2010, 3342],
 [2010, 3342, 3807],
 [3342, 3807, 768],
 [3807, 768, 1171],
 [768, 1171, 32],
 [1171, 32, 43],
 [32, 43, 250],
 [43, 250, 926],
 [250, 926, 2],
 [1, 47, 5142],
 [47, 5142, 93],
 [5142, 93, 232],
 [93, 232, 77],
 [232, 77, 328],
 [77, 328, 84],
 [328, 84, 848],
 [84, 848, 2195],
 [848, 2195, 116],
 [2195, 116, 41],
 [116, 41, 5143],
 [41, 5143, 11055],
 [5143, 11055, 1318],
 [11055, 1318, 350],
 [1318, 350, 5144],
 [350, 5144, 3343],
 [5144, 3343, 1734],
 [3343, 1734, 881],
 [1734, 881, 3808],
 [881, 3808, 2],
 [1, 175, 573],
 [175, 573, 6277],
 [573, 6277, 1319],
 [6277, 1319, 164],
 [1319, 164, 103],
 [164, 103, 2196],
 [103, 2196, 6278],
 [2196, 6278, 980],
 [6278, 980, 97],
 [980, 97, 1402],
 [97, 1402, 4407],
 [1402, 4407, 882],
 [4407, 882, 11056],
 [882, 11056, 13],
 [11056, 13, 329],
 [13, 329, 3344],
 [329, 3344, 5145],
 [3344, 5145, 11057],
 [5145, 11057, 11058],
 [11057, 11058, 11059],
 [11058, 11059, 2],
 [1, 382, 151],
 [382, 151, 84],
 [15

### c) Then, split the sequences into X and y and create a Data Generator

In [12]:
# Note here that the sequences were in the form: 
# sequence = [x1, x2, ... , x(n-1), y]
# We still need to separate it into [[x1, x2, ... , x(n-1)], ...], [y1, y2, ...]

def generate_xy(encoded: list, ngram: int):
    training_samples = generate_ngram_training_samples(encoded, ngram)
    toreturn_X = []
    toreturn_y = []
    random.shuffle(training_samples)
    for sample in training_samples:
        toreturn_X.append(sample[:-1])
        toreturn_y.append(sample[-1])
    return toreturn_X, toreturn_y

In [14]:
def read_embeddings(filename, tokenizer):
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    '''
    
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':[0....], ...}
    # index to embedding : {1:[0....], ...} 
    # use your tokenizer's word_index to find the index of
    # a given word
    
    # code to read the embeddings file and return the above two dicts
    f = open(filename, 'r')
    f.readline()
    word_to_embedding = {}
    index_to_embedding = {}
    for line in f:
        line = line.split()
        word_to_embedding[line[0]] = [float(x) for x in line[1:]]
        #index to embedding refer to the index of the word in the tokenizer
        index_to_embedding[tokenizer.word_index[line[0]]] = [float(x) for x in line[1:]]
    return word_to_embedding, index_to_embedding




In [15]:
def data_generator(X: list, y: list, num_sequences_per_batch: int, embedding: dict) -> (list,list):
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    '''
    a = []
    b = []
    for i in range(len(X)):
        temp = []
        for item in X[i]:
            temp.extend(embedding[item])
        a.append(temp)
        b.append(to_categorical(y[i], num_classes= len(embedding) + 1))
        if len(a) == num_sequences_per_batch:
            yield (np.matrix(a), np.matrix(b))
            a = []
            b = []
    yield (a, b)

In [16]:
X, y = generate_xy(encoded, NGRAM)
X

[[1, 3835],
 [449, 688],
 [1, 47],
 [16403, 511],
 [4559, 2091],
 [2064, 3654],
 [211, 13853],
 [16, 2505],
 [404, 767],
 [253, 5575],
 [77, 328],
 [1, 174],
 [20, 90],
 [10658, 359],
 [463, 463],
 [1, 84],
 [444, 585],
 [3711, 149],
 [119, 1523],
 [2691, 3304],
 [10271, 3942],
 [9884, 232],
 [2872, 6540],
 [149, 11862],
 [1, 720],
 [4174, 650],
 [581, 1667],
 [1665, 2685],
 [1268, 5111],
 [21, 47],
 [1046, 5071],
 [41, 5788],
 [1363, 1807],
 [11318, 8214],
 [13669, 30],
 [368, 2858],
 [717, 3226],
 [1, 1351],
 [29, 263],
 [1929, 342],
 [27, 4124],
 [4009, 123],
 [1930, 207],
 [1604, 512],
 [13079, 5920],
 [670, 1025],
 [5127, 18293],
 [7749, 7321],
 [1628, 5028],
 [569, 99],
 [1254, 827],
 [48, 8240],
 [236, 99],
 [1228, 14],
 [7275, 3296],
 [240, 105],
 [112, 5604],
 [179, 163],
 [4649, 50],
 [10845, 10846],
 [8535, 3277],
 [786, 2727],
 [105, 3990],
 [2384, 19],
 [1, 63],
 [15600, 86],
 [1055, 9533],
 [1727, 167],
 [4662, 841],
 [206, 2068],
 [6952, 5678],
 [1, 12],
 [1092, 1902],
 

In [17]:
tokenizer.word_index

{'<s>': 1,
 '</s>': 2,
 's': 3,
 'me': 4,
 'man': 5,
 'said': 6,
 'it': 7,
 'time': 8,
 'like': 9,
 'old': 10,
 'eyes': 11,
 'night': 12,
 'life': 13,
 'day': 14,
 'long': 15,
 'saw': 16,
 'little': 17,
 'him': 18,
 'great': 19,
 'came': 20,
 'and': 21,
 'thought': 22,
 'heard': 23,
 'death': 24,
 'i': 25,
 'mind': 26,
 'far': 27,
 'things': 28,
 'house': 29,
 'men': 30,
 'heart': 31,
 'thing': 32,
 'earth': 33,
 'left': 34,
 'years': 35,
 'you': 36,
 'love': 37,
 'shall': 38,
 'world': 39,
 'light': 40,
 'way': 41,
 'know': 42,
 'place': 43,
 'felt': 44,
 'that': 45,
 'however': 46,
 'having': 47,
 'seen': 48,
 'room': 49,
 'human': 50,
 'length': 51,
 'raymond': 52,
 'strange': 53,
 'head': 54,
 'come': 55,
 'voice': 56,
 'friend': 57,
 'knew': 58,
 'door': 59,
 'new': 60,
 'words': 61,
 'her': 62,
 'moment': 63,
 'away': 64,
 'father': 65,
 'let': 66,
 'half': 67,
 'hand': 68,
 'good': 69,
 't': 70,
 'soon': 71,
 'them': 72,
 'near': 73,
 'nature': 74,
 'sea': 75,
 'body': 76,
 'too

In [19]:
# Examples
# initialize data_generator
# num_sequences_per_batch = 128 # this is the batch size
# steps_per_epoch = len(sequences)//num_sequences_per_batch  # Number of batches per epoch
# train_generator = data_generator(X, y, num_sequences_per_batch)

# sample=next(train_generator) # this is how you get data out of generators
# sample[0].shape # (batch_size, (n-1)*EMBEDDING_SIZE)  (128, 200)
# sample[1].shape   # (batch_size, |V|) to_categorical

w, i = read_embeddings('half_spooky_embeddings.txt', tokenizer)
X, y = generate_xy(encoded, NGRAM)
num_sequences_per_batch = 128 # this is the batch size
steps_per_epoch = len(X)//num_sequences_per_batch  # Number of batches per epoch
train_generator = data_generator(X, y, num_sequences_per_batch, i)
sample=next(train_generator) # this is how you get data out of generators
print(sample[0].shape) # (batch_size, (n-1)*EMBEDDING_SIZE)  (128, 200)
print(sample[1].shape) # (batch_size, |V|) to_categorical

(128, 400)
(128, 19034)


### d) Train your models

In [20]:
# code to train a feedforward neural language model 
# on a set of given word embeddings
# make sure not to just copy + paste to train your two models

# Define the model architecture using Keras Sequential API

def feed_forward(embedding_size: int, vocab_size: int, ngram: int) -> Sequential:
    '''
    Returns a feed forward neural network model
    '''
    learning_rate = 0.1
    model = Sequential()
    model.add(Dense(256, input_dim=(ngram - 1) * embedding_size, activation='relu'))
    model.add(Dense(vocab_size, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=learning_rate), metrics=['accuracy'])
    return model
# code to train the model
model = feed_forward(EMBEDDINGS_SIZE, len(tokenizer.word_index) + 1, NGRAM)


2023-03-21 19:30:44.003390: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [21]:

# Start training the model
model.fit(x=train_generator, 
          steps_per_epoch=steps_per_epoch,
          epochs=10)
model

Epoch 1/10
938/938 [==============================] - 57s 60ms/step - loss: 8.9993 - accuracy: 0.0812
Epoch 2/10
938/938 [==============================] - 0s 478us/step - loss: 8.9993 - accuracy: 0.0812


### e) Generate Sentences

In [22]:
# generate a sequence from the model
def generate_seq(model: Sequential, 
                 tokenizer: Tokenizer, 
                 seed: list, 
                 n_words: int):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
        n_words: generate a sentence of length n_words
    Returns: string sentence
    '''
    sentence = seed
    pres = seed
    word_range = [i for i in range(len(tokenizer.word_index) + 1)]
    for _ in range(n_words):
        input_X = []
        for item in pres:
            input_X += w[item]
        input_X = np.matrix(input_X)
        output_y = model.predict(input_X)
        #randomly choose the next word
        output_y = list(list(output_y)[0])
        word_chosen = random.choices(word_range, output_y)[0]
        sentence += [tokenizer.index_word[word_chosen]]
        if tokenizer.index_word[word_chosen] == '</s>':
            print("true")
            return sentence
        pres = sentence[-NGRAM + 1:]
    return ' '.join(sentence)

In [24]:
generate_seq(model, tokenizer, ['<s>', '<s>'], 20)

1/1 [==============================] - 0s 22ms/step
true


['<s>',
 '<s>',
 'terrible',
 'beauty',
 'transports',
 'exploit',
 'doch',
 'buoys',
 'distinguish',
 'design',
 'penetrate',
 'deepened',
 'foretopsail',
 'isles',
 'nose',
 '</s>']

### f) Compare your generated sentences

You may find it useful to run your HW 2 code on one of the datasets (or a subset of the dataset) that you used for this homework.

Sources Cited
----------------------------
